# Factorization Machines (FM)

Factorization machines (FM) model all nested interactions up to order $d$ between the $p$ input variables in $x$ using factorized interaction parameters. The factorization machine (FM) model is defined as

$$\hat{y}(\text{x})= w_0 + \sum_{j=1}^d w_j x_j + \sum_{j=1}^d \sum_{j'=j+1}^d x_j x_{j'} < v_j, v_{j'} > $$

The first part corresponds to linear model, the second part contains all pairwise interactions between input variables. The important difference to standard polynomial regression is that the effect of the interaction is not modeled by an independent parameter $w_{j, j'}$ but with a factorized parametrization $w_{j, j'} = <v_j, v_{j'}>=\sum_{f=1}^k v_{j,f}v_{j',f}$, which corresponds to the assumption that the effect of pairwise interactions has a low rank.

Three learning methods (for minimization of given loss function $\mathcal{L}$) are widely used for FMs:
- stochastic gradient descent (SGD) and modifications
- alternating least-squares (ALS)
- and Markov Chain Monte Carlo (MCMC) inference

In [1]:
import numpy
import astropy
from astropy.io import ascii
from astropy.table import Table
from sklearn.metrics import roc_auc_score

# fastFM is python-friendly implementation of factorization machines
import fastFM

### Read data for flight delay challenge

In [2]:
data = ascii.read("data/training.csv", delimiter=',')  
test_kaggle = ascii.read("data/test.csv", delimiter=',')  

binary_target = (data['ARRIVAL_DELAY'] > 10) * 1
data.remove_column('ARRIVAL_DELAY')

# take small part of data
small_data = data[::20]
small_binary_target = binary_target[::20]

### helping functions from the first notebook

In [3]:
def train_test_split(*arrays, **kargs):
    '''modification of sklearn's train_test_split to support astropy. See sklearn documentation for parameters '''
    from sklearn.cross_validation import train_test_split
    arrays2 = map(lambda x: numpy.array(x) if isinstance(x, Table) else x, arrays)
    results = list(train_test_split(*arrays2, **kargs))
    
    for i in range(len(results) // 2):
        if isinstance(arrays[i], Table):
            results[2 * i] = Table(results[2 * i])
            results[2 * i + 1] = Table(results[2 * i + 1])
    return results

In [4]:
from IPython.display import FileLink

def create_solution(predictions, filename='flight-delay-predictions.csv'):
    result = astropy.table.Table({'ID': numpy.arange(len(predictions)), 'ARRIVAL_DELAY': predictions})
    result.write('data/{}'.format(filename), format='csv', delimiter=',', overwrite=True)
    return FileLink('data/{}'.format(filename))

In [5]:
non_categorical_features = ['SCHEDULED_DEPARTURE', 'DISTANCE', 'SCHEDULED_ARRIVAL']
categorical_features = list(set(data.columns) - set(non_categorical_features))

In [6]:
# prepare training and test samples
trainX, testX, trainY, testY = train_test_split(small_data, small_binary_target, random_state=42, train_size=0.5)

# normalize non categorical features for optimization process stability
for column in non_categorical_features:
    coeff = numpy.std(trainX[column])
    testX[column]  = testX[column] / coeff
    trainX[column] = trainX[column] / coeff
    # prepare test kaggle samples
    test_kaggle[column] = test_kaggle[column] / coeff

/Users/antares/.virtualenvs/rep/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### Prepare data for factorization machines
one-hot encoding is used

In [7]:
from sklearn.preprocessing import OneHotEncoder
coder = OneHotEncoder(sparse=True, categorical_features=numpy.in1d(trainX.colnames, categorical_features),
                      handle_unknown='ignore')
coder.fit(trainX.to_pandas())

OneHotEncoder(categorical_features=array([ True,  True,  True,  True,  True,  True,  True,  True, False,
       False, False], dtype=bool),
       dtype=<type 'numpy.float64'>, handle_unknown='ignore',
       n_values='auto', sparse=True)

In [8]:
# transform categorical features into vectors with zeros and ones
trainX_sparse = coder.transform(trainX.to_pandas())
testX_sparse = coder.transform(testX.to_pandas())

test_kaggle_sparse = coder.transform(test_kaggle.to_pandas())

## ALS 

Alternating least-squares (ALS) is a coordinate descent method, frequently used in factorization models.

<!--
The optimization approach of SGD is based on iterating over cases (rows) of the training data and performing small steps in the direction of a smaller loss. Coordinate descent or alternating least-squares (ALS) takes another approach by minimizing the loss per model parameter: we iterate over parameters, fix all parameters except the current parameter $\theta^*$, find $\hat{\theta}^*=\arg\min_{\theta^*} \mathcal L$ and use it as estimation of parameter $\theta^*$.
-->

In [9]:
# import model, it follows sklearn interface
from fastFM.als import FMClassification as FMClassificationALS

fm = FMClassificationALS(n_iter=100)
# target should be coded as {-1, 1}
fm.fit(trainX_sparse, trainY * 2 - 1)
proba_als = fm.predict_proba(testX_sparse)

In [10]:
roc_auc_score(testY, proba_als)

0.54313173055361774

Regularization and initialization of parameters (they are initialized with normal distribution) play significant role in FM training:

In [11]:
fm = FMClassificationALS(n_iter=50, l2_reg_V=5, l2_reg_w=5, init_stdev=0.01)
# target should be coded as {-1,1}
fm.fit(trainX_sparse, trainY * 2 - 1)
roc_auc_score(testY, fm.predict_proba(testX_sparse))

0.60078186815056023

## Averaging for different regularizations

In [12]:
proba = numpy.zeros(len(testY))
for reg_V in [0.1, 0.5, 1, 2, 3, 4, 5, 6, 7, 8]:
    fm = FMClassificationALS(n_iter=50, l2_reg_V=reg_V, l2_reg_w=5, init_stdev=0.01)
    # target should be coded as {-1,1}
    fm.fit(trainX_sparse, trainY * 2 - 1)
    proba += fm.predict_proba(testX_sparse)
    print roc_auc_score(testY, proba)

0.555581409726
0.575374847872
0.584520700631
0.591473818172
0.595013148803
0.59923225186
0.603069565927
0.605306563354
0.606520811103
0.607179476674


## MCMC

Both ALS and SGD learn the best parameters which are used for a point estimate of $\hat{y}$. MCMC is a Bayesian
inference technique that generates the distribution of $\hat{y}$ by sampling. There is a [demo](http://arogozhnikov.github.io/2016/12/19/markov_chain_monte_carlo.html) with playground which explains the MCMC approach.

In [13]:
from fastFM.mcmc import FMClassification as FMClassificationMCMC
fm = FMClassificationMCMC(n_iter=100)
# keeping all models takes too much space, so predictions are accumulated in the train-time
proba_mcmc = fm.fit_predict_proba(trainX_sparse, trainY * 2 - 1, testX_sparse)

In [14]:
roc_auc_score(testY, proba_mcmc)

0.63711272853109091

In [15]:
proba_mcmc_kaggle = fm.fit_predict_proba(trainX_sparse, trainY * 2 - 1, test_kaggle_sparse)
create_solution(proba_mcmc_kaggle, 'flight-delay-mcmc.csv')

/Users/antares/Yandex.Disk.localized/2017-04-Heidelberg/practice/data/flight-delay-mcmc.csv

## Simple mixing with GB

As soon as FM and GB are very different model, mixing of two models is expected to be better than separate models. Let's try:

In [16]:
from xgboost.sklearn import XGBClassifier

xgb_clf = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=4)
xgb_clf.fit(trainX.to_pandas(), trainY)
proba_gb = xgb_clf.predict_proba(testX.to_pandas())[:, 1]
roc_auc_score(testY, proba_gb)

0.64552997411209456

Simple averaging of two models

In [17]:
roc_auc_score(testY, (proba_mcmc + proba_gb) / 2.)

0.65215540498275426

<!-- ## Stacking (stacked generalization)

<img src='http://arogozhnikov.github.io/images/etc/rogozhnikov_stacking.png' />
-->

###  You can do better!
- Increase number of samples
- Play with regularization in FM model: regularization is the most important in FM training
- Feature engineering matters for FMs too!
- Mix up different models using also machine learning?

# Neural Networks & pytorch

There are many different libraries for deep learning, we'll use [pytorch](http://pytorch.org), a fastest to dive library that benefits from torch infrastructure and [chainer](http://chainer.org) approach 'define-by-run'.

> Warning: `pytorch` is unstable, not even beta.

### Autograd 101

The main point of NN training is gradient computation with respect to parameters (weights). 

Consider the expression 
$$ f = <w, x> = \sum_j x_j w_j$$
and the gradient with respect to $w$
$$ \frac{\partial{f}}{\partial{w_i}} = x_i$$
or in vector form
$$ \frac{\partial{f}}{\partial{w}} = x$$

Let see how this is done in `pytorch`.

In [18]:
import torch
from torch.autograd import Variable
dtype = torch.FloatTensor

In [19]:
# create sample with shape [1, 10] and set that gradients are not needed for this variable
x = Variable(torch.randn(1, 10).type(dtype), requires_grad=False)
# create weights vector with shape [10, 1] and set that we want to compute gradients with respect to this varibale
w = Variable(torch.randn(10, 1).type(dtype) / 2, requires_grad=True)

In [20]:
# print values of x and w
print x.data
print w.data


-0.6530 -1.6536 -0.9376  1.0105 -1.0674  1.4680  0.5759  0.4326 -0.0700 -0.4823
[torch.FloatTensor of size 1x10]


 0.1229
 0.4837
 0.5096
-0.4697
 0.2409
 1.3351
 0.2483
 0.8510
 0.1855
 0.5166
[torch.FloatTensor of size 10x1]



In [21]:
# convert to numpy
print x.data.numpy(), x.data.numpy().shape

[[-0.65301806 -1.6536448  -0.93764013  1.01047695 -1.06735981  1.46797395
   0.57594097  0.43258277 -0.0699688  -0.4823184 ]] (1, 10)


In [22]:
# compute expression (as in numpy), where mm is matrix multiplication
f = x.mm(w)
# Use autograd to compute the backward pass. This call will compute the
# gradient of f with respect to all Variables with requires_grad=True.
f.backward()
# call w.grad will be Variables holding the gradient
# of the loss with respect to w.
w.grad.data


-0.6530
-1.6536
-0.9376
 1.0105
-1.0674
 1.4680
 0.5759
 0.4326
-0.0700
-0.4823
[torch.FloatTensor of size 10x1]

Indeed, gradient is equal to $x$.

## A bit harder

Compute gradients with respect to $w_1$:

$$\mathcal{L} = \sum_i \dfrac{\sin{(e^{(2 w_{1, i} - 1) x_i})}}{3}$$

for $x = (1, .., 1)\in \mathbb{R}^{10}$

In [23]:
x = Variable(torch.ones(10).type(dtype), requires_grad=False)
w1 = Variable(torch.FloatTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), requires_grad=True)

w2 = 2 * w1 - 1
w3 = torch.exp(w2 * x)
w4 = torch.sin(w3) / 3
loss = w4.sum()
loss.backward()

In [24]:
w1.grad

Variable containing:
-1.6522e+00
 4.4000e+00
-7.1832e+01
-7.1389e+02
-3.2876e+03
-3.8672e+03
-5.6167e+04
-6.0108e+04
 1.6065e+07
 1.0951e+08
[torch.FloatTensor of size 10]

### Question: how was gradient computed?

<details>
 <summary>Answer</summary>
 <p>
 $$ \mathcal{L} = \mathcal{L}(w_5) = \mathcal{L}(w_5 (w_4)) = \dots = \mathcal{L}(w_5 (w_4 (w_3 (w_2 (w_1)))))
 $$
 </p>
 <p>
     Computations (forward propagation) are done in the following order:
     $$
         w_1 \rightarrow w_2 \rightarrow w_3 \rightarrow w_4 \rightarrow w_5
     $$
 </p>
 <p>
     Computation of gradient:
 </p>
 <p>
     \begin{aligned}
         \dfrac{ \partial \mathcal{L}}{\partial w_5} & \text{is straightforward computation} \\
         \dfrac{ \partial \mathcal{L}}{\partial w_4} &= \dfrac{ \partial \mathcal{L}}{\partial w_5} \dfrac{ \partial \mathcal{w_5}}{\partial w_4} \\
         \dfrac{ \partial \mathcal{L}}{\partial w_3} &= \dfrac{ \partial \mathcal{L}}{\partial w_4} \dfrac{ \partial \mathcal{w_4}}{\partial w_3} \\
         \dfrac{ \partial \mathcal{L}}{\partial w_2} &= \dfrac{ \partial \mathcal{L}}{\partial w_3} \dfrac{ \partial \mathcal{w_3}}{\partial w_2} \\
         \dfrac{ \partial \mathcal{L}}{\partial w_1} &= \dfrac{ \partial \mathcal{L}}{\partial w_2} \dfrac{ \partial \mathcal{w_2}}{\partial w_1} \\
     \end{aligned}
 </p>
 <p>
     These computations are done in reverse order 
     $$ \dfrac{ \partial \mathcal{L}}{\partial w_1}  \leftarrow 
         \dfrac{ \partial \mathcal{L}}{\partial w_2} \leftarrow 
         \dfrac{ \partial \mathcal{L}}{\partial w_3} \leftarrow 
         \dfrac{ \partial \mathcal{L}}{\partial w_4} \leftarrow 
         \dfrac{ \partial \mathcal{L}}{\partial w_5}  
     $$ 
     this process is called `backward propagation` or just `backpropagation`.
 </p>
</details>

In [25]:
# Let see the backpropagation
def printgrad(name):
    def print_hook(grad):
        print name, grad 
    return print_hook
    
x = Variable(torch.ones(10).type(dtype), requires_grad=False)
w1 = Variable(torch.FloatTensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]), requires_grad=True)
w1.register_hook(printgrad('w1'))
w2 = 2 * w1 - 1
w2.register_hook(printgrad('w2'))
w3 = torch.exp(w2 * x)
w3.register_hook(printgrad('w3'))
w4 = torch.sin(w3) / 3
w4.register_hook(printgrad('w4'))
loss = w4.sum()


# first the gradient over w4 is printed, then over w3, then over w2, and then over w1
loss.backward()

w4 Variable containing:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
[torch.FloatTensor of size 10]

w3 Variable containing:
-0.3039
 0.1095
-0.2420
-0.3255
-0.2029
-0.0323
-0.0635
-0.0092
 0.3325
 0.3068
[torch.FloatTensor of size 10]

w2 Variable containing:
-8.2612e-01
 2.2000e+00
-3.5916e+01
-3.5694e+02
-1.6438e+03
-1.9336e+03
-2.8084e+04
-3.0054e+04
 8.0326e+06
 5.4753e+07
[torch.FloatTensor of size 10]

w1 Variable containing:
-1.6522e+00
 4.4000e+00
-7.1832e+01
-7.1389e+02
-3.2876e+03
-3.8672e+03
-5.6167e+04
-6.0108e+04
 1.6065e+07
 1.0951e+08
[torch.FloatTensor of size 10]



## Simple NN

Let code the simple NN with one hidden layer with `pytorch`

Parameters:
- $W$, $v$

Calculations:
- hidden activations: $h_{ik} = \sigma(\sum_j X_{ij} W_{jk} )$
- $p_i$ = $\sigma(\sum_k h_{ik} v_{k} )   $
- loss function can be written as
  $$\mathcal{L}=-\sum_i y_i \log{p_i} + (1-y_i)\log{(1 - p_i)}\qquad,$$ where $y \in \{0, 1\}$
- compute loss function gradient with respect to parameters


### Prepare data for NNs with a `StandardScaler`

In [26]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(trainX.to_pandas())
trainX_scaled = scaler.transform(trainX.to_pandas())
testX_scaled = scaler.transform(testX.to_pandas())

### Training

In [27]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(torch.FloatTensor(trainX_scaled), torch.FloatTensor(trainY.reshape((len(trainY), 1))))
loader_train = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

In [28]:
dtype = torch.FloatTensor

# batch_size is batch size; dim_input is input dimension;
# dim_hidden is hidden dimension; dim_output is output dimension.
batch_size, dim_input, dim_hidden, dim_output = 64, len(trainX.colnames), 15, 1

# Create random Tensors for weights, and wrap them in Variables.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Variables during the backward pass.
w = Variable(torch.randn(dim_input, dim_hidden).type(dtype), requires_grad=True)
v = Variable(torch.randn(dim_hidden, dim_output).type(dtype), requires_grad=True)

learning_rate = 1e-4
for iteration in range(100):
    # use SGD optimization and iterate over batch
    average_loss = 0
    for nbatch, (batch_X, batch_y) in enumerate(loader_train):
        batch_X, batch_y = Variable(batch_X), Variable(batch_y)
        
        # Forward pass: compute predicted y using operations on Variables;
        hidden_activations = batch_X.mm(w).sigmoid()
        y_pred = hidden_activations.mm(v).sigmoid()
        # Compute and print loss using operations on Variables.
        # Now loss is a Variable of shape (1,) and loss.data is a Tensor of shape
        # (1,); loss.data[0] is a scalar value holding the loss.
        loss = - (batch_y * torch.log(y_pred) + (1 - batch_y) * torch.log(1 - y_pred)).sum()
        average_loss += loss.data[0]
        
        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Variables with requires_grad=True.
        # After this call w.grad and v.grad will be Variables holding the gradient
        # of the loss with respect to w and v respectively.
        loss.backward()

        # Update weights using gradient descent; w1.data and w2.data are Tensors,
        # w1.grad and w2.grad are Variables and w1.grad.data and w2.grad.data are
        # Tensors.
        w.data -= learning_rate * w.grad.data
        v.data -= learning_rate * v.grad.data

        # Manually zero the gradients after updating weights
        w.grad.data.zero_()
        v.grad.data.zero_()
    if iteration % 5 == 0:
        print iteration, '\t', average_loss / len(trainY)

0 	0.706803307415
5 	0.58766642908
10 	0.556104045263
15 	0.541402690094
20 	0.534092096746
25 	0.530062030943
30 	0.527607891954
35 	0.525937170661
40 	0.524670135692
45 	0.523656870161
50 	0.522795454201
55 	0.522036619202
60 	0.52134101868
65 	0.520706337014
70 	0.520127066595
75 	0.519583558773
80 	0.519081023461
85 	0.518599956505
90 	0.518144798859
95 	0.517732902518


### Predict test samples

In [29]:
testX_var = Variable(torch.FloatTensor(testX_scaled))
proba_var = testX_var.mm(w).sigmoid().mm(v).sigmoid()
roc_auc_score(testY, proba_var.data.numpy())

0.58307875856574343

**Exercise:** how to modify previous code to have SGD with momentum?

## Simple NN with layers and optimization methods

All the needed components are written already for us, so let's just use them.

In [30]:
batch_size, dim_input, dim_hidden, dim_output = 64, len(trainX.colnames), 15, 1

# Use the nn package to define our model and loss function.
# sequence of layers
model = torch.nn.Sequential(
    torch.nn.Linear(dim_input, dim_hidden),
    torch.nn.Sigmoid(),
    torch.nn.Linear(dim_hidden, dim_output),
)
# define logistic loss (take NN output, apply sigmoid and compute logistic loss)
loss_fn = torch.nn.SoftMarginLoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam optimization methos; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for iteration in range(20):
    average_loss = 0
    # use optimization over batch
    for nbatch, (batch_X, batch_y) in enumerate(loader_train):
        batch_X, batch_y = Variable(batch_X), Variable(batch_y)
        # loss function takes {-1, 1} as target
        batch_y = 2 * batch_y - 1
        
        # Forward pass: compute predicted y by passing x to the model.
        pred = model(batch_X)

        # Compute and print loss.
        loss = loss_fn(pred, batch_y)
        average_loss += loss.data[0]

        # Before the backward pass, use the optimizer object to zero all of the
        # gradients for the variables it will update 
        # (which are the learnable weights of the model)
        optimizer.zero_grad()

        # Backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()

        # Calling the step function on an Optimizer makes an update of parameters
        optimizer.step()
    print iteration, average_loss / len(trainX)

0 0.66745367231
1 0.523617102622
2 0.511197137234
3 0.509856051213
4 0.509322984625
5 0.509123538793
6 0.508960434119
7 0.508864008313
8 0.508786980206
9 0.508743424029
10 0.508654685386
11 0.508629862152
12 0.508581621509
13 0.508713238715
14 0.50860274833
15 0.508476358781
16 0.508499256466
17 0.508510646376
18 0.508560741483
19 0.50847081916


In [31]:
from torch.nn.functional import sigmoid

testX_var = Variable(torch.FloatTensor(testX_scaled))
proba_var = sigmoid(model(testX_var))
roc_auc_score(testY, proba_var.data.numpy())

0.60418980220645291

# References
- [Paper](https://www.csie.ntu.edu.tw/~b97053/paper/Factorization%20Machines%20with%20libFM.pdf) on factorization machines
- [Pytorch documentation](http://pytorch.org/tutorials/index.html)